0820-Copy1が全ての雛形です。  
201027textmining_3_hypertuning  
201110_MGI追加のコピー  
210816 catboost


#gene_to_symの作成

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/nlp
import pandas as pd
import numpy as np
import pickle

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/nlp


In [ ]:
!head HGNC_0908.txt

Approved symbol	Approved name	Previous symbols	Alias symbols	Chromosome	Alias names	Previous name
A1BG	alpha-1-B glycoprotein			19q13.43		
A1BG-AS1	A1BG antisense RNA 1	NCRNA00181, A1BGAS, A1BG-AS	FLJ23569	19q13.43		"non-protein coding RNA 181", "A1BG antisense RNA (non-protein coding)", "A1BG antisense RNA 1 (non-protein coding)"
A1CF	APOBEC1 complementation factor		ACF, ASP, ACF64, ACF65, APOBEC1CF	10q11.23		
A1S9T	symbol withdrawn, see [HGNC:12469](/data/gene-symbol-report/#!/hgnc_id/HGNC:12469)					
A2M	alpha-2-macroglobulin		FWP007, S863-7, CPAMD5	12p13.31		
A2M-AS1	A2M antisense RNA 1			12p13.31		"A2M antisense RNA 1 (non-protein coding)", "A2M antisense RNA 1"
A2ML1	alpha-2-macroglobulin like 1	CPAMD9	FLJ25179, p170	12p13.31		"C3 and PZP-like, alpha-2-macroglobulin domain containing 9"
A2ML1-AS1	A2ML1 antisense RNA 1			12p13.31		"A2ML1 antisense RNA 1 (non-protein coding)"
A2ML1-AS2	A2ML1 antisense RNA 2			12p13.31		"A2ML1 antisense RNA 2 (non-protein coding)"


In [ ]:
ju[[type(a) == float for a in ju["Approved symbol"]]]

,Approved symbol,Approved name,Previous symbols,Alias symbols,Chromosome,Alias names,Previous name


In [ ]:
import pandas as pd
#ju = pd.read_table("./HGNC_0908.txt")#HGNCからダウンロードしたものを保存
#ju = ju[[type(a) != float for a in ju["Approved symbol"]]]
#ju = ju[["withdrawn" not in a for a in ju["Approved name"]]]
#sym = list(ju["Approved symbol"])
#asy = list(ju["Alias symbols"])
#nam = list(ju["Approved name"])
#ana = list(ju["Alias names"])
#pna = list(ju["Previous name"])
#psy = list(ju["Previous symbols"])
#chr = list(ju["Chromosome"])

sym,asy,nam,ana,pna,psy,chr = [],[],[],[],[],[],[]
f = open("./HGNC_0908.txt","r")
for line in f:
  if line.startswith("Approved"):continue
  ls = line.strip("\n").split("\t")
  sym.append(ls[0])
  nam.append(ls[1])
  psy.append(ls[2])
  asy.append(ls[3])
  chr.append(ls[4])
  try:ana.append(ls[5])
  except:print(ls)
  pna.append(ls[6])
f.close()

gene_to_sym={}
for n,s in enumerate(sym):
  gene_to_sym[s.upper()] = s.upper()
ch = set([s.upper() for s in sym])
du = set()

def geun_na(col,g_t_u,d):
  for n in range(len(col)):
      a = col[n]
      if type(a) == float:continue#None
      a = a.upper()
      ts = a.split("\",")
      for aa in ts:
        aa = aa.replace("\"","")
        #if "(" in aa:
        #  aa = aa[:aa.index("(")-1]
        aa = aa.strip()
        if aa in ch:continue
        if aa in g_t_u:
          d.add(aa)
          continue
        g_t_u[aa]=sym[n]
  return g_t_u,d

def geun(col,g_t_u,d):
  for n in range(len(col)):
      a = col[n]
      if type(a) == float:continue#None
      a = a.upper()
      for aa in a.split(','):
        #aa = aa.replace("\"","")
        #if "(" in aa:
        #  aa = aa[:aa.index("(")-1]
        aa = aa.strip()
        if aa in ch:continue
        if aa in g_t_u:
          d.add(aa)
          continue
        g_t_u[aa]=sym[n]
  return g_t_u,d

gene_to_sym,du = geun(asy,gene_to_sym,du)
gene_to_sym,du = geun_na(nam,gene_to_sym,du) 
gene_to_sym,du = geun_na(ana,gene_to_sym,du) 
gene_to_sym,du = geun_na(pna,gene_to_sym,du) 
gene_to_sym,du = geun(psy,gene_to_sym,du) 
for d in du:
  del gene_to_sym[d]
gene_to_sym2 = {}
for g in gene_to_sym:#haihun処理
  gene_to_sym2[g] = gene_to_sym[g]
  if "-" in g and g.replace("-","") not in gene_to_sym:
    gene_to_sym2[g.replace("-","")] = gene_to_sym[g]
gene_to_sym = gene_to_sym2

In [ ]:
for g in gene_to_sym:
  if gene_to_sym[g]=="A1BG-AS1":
    print(g)

A1BG-AS1
A1BGAS1
FLJ23569
A1BG ANTISENSE RNA 1
NON-PROTEIN CODING RNA 181
NONPROTEIN CODING RNA 181
A1BG ANTISENSE RNA (NON-PROTEIN CODING)
A1BG ANTISENSE RNA (NONPROTEIN CODING)
A1BG ANTISENSE RNA 1 (NON-PROTEIN CODING)
A1BG ANTISENSE RNA 1 (NONPROTEIN CODING)
NCRNA00181
A1BGAS
A1BG-AS


In [ ]:
gene_to_sym["ART"]

'AGRP'

In [ ]:
import pickle
with open("gene_to_sym_0919.pickle", "wb") as f:
  pickle.dump(gene_to_sym,f)

In [ ]:
pna[:10]


[nan,
 'non-protein coding RNA 181, "A1BG antisense RNA (non-protein coding)", "A1BG antisense RNA 1 (non-protein coding)"',
 nan,
 nan,
 'A2M antisense RNA 1 (non-protein coding), "A2M antisense RNA 1"',
 'C3 and PZP-like, alpha-2-macroglobulin domain containing 9',
 'A2ML1 antisense RNA 1 (non-protein coding)',
 'A2ML1 antisense RNA 2 (non-protein coding)',
 'alpha-2-macroglobulin pseudogene',
 'alpha 1,3-galactosyltransferase 2, pseudogene']

#特徴量作成


In [ ]:
#chromosome
chrdic = {}
chrset = set()
for n,s in enumerate(sym):
  s = s.upper()
  if type(chr[n]) == float:continue
  if "p" in chr[n]:
     c = chr[n][:chr[n].index("p")+1]
  elif "q" in chr[n]:
     c = chr[n][:chr[n].index("q")+1]
  else:continue
  if len(c) > 4:continue
  if c == "unp":continue
  chrset.add(c)
  chrdic[s]=set([c])
chrlist = sorted(list(chrset))
chrnum = {}
for n,e in enumerate(chrlist):
  chrnum[e] = n

In [ ]:
chrset

In [ ]:
#!wget http://geneontology.org/gene-associations/goa_human.gaf.gz
#!gunzip goa_human.gaf.gz
dfgo = pd.read_table("goa_human.gaf",skiprows=41,header=None)
ggdic = {}
name = dfgo[2].tolist()
goid = dfgo[4].tolist()
goset= set(goid)
no=set()
for n in range(len(dfgo)):
  nu = name[n].upper()
  if nu in gene_to_sym:
    u = gene_to_sym[nu]
    if u not in ggdic:ggdic[u]=set()
    ggdic[u].add(goid[n])
  else:no.add(nu)
golist = sorted(list(goset))
gonum2 = {}
for n in range(len(golist)):
   gonum2[golist[n]]=n

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
#mgi
f = open("HMD_HumanPhenotype.rpt","r")
mgidic = {}
mgiset = set()
for line in f:
  line_s = line.strip().split("\t")
  if len(line_s) >= 7:
      if line_s[0].upper() not in gene_to_sym:
        continue
      mgidic[gene_to_sym[line_s[0].upper()]]= set(line_s[6].split(", "))
      for n in line_s[6].split(", "):
        mgiset.add(n)
  else:
    if line_s[0].upper() not in gene_to_sym:
        continue
    mgidic[gene_to_sym[line_s[0].upper()]] = set()
mgilist = list(mgiset)
mgilist.sort()
mginum = {}
for t in range(len(mgilist)):
    mginum[mgilist[t]] =t
f.close()

In [ ]:
#HPO,OMIM,ORPHAN
#!wget http://compbio.charite.de/jenkins/job/hpo.annotations/lastSuccessfulBuild/artifact/util/annotation/genes_to_phenotype.txt

f = open("genes_to_phenotype.txt", "r")
a = 0
hpodic = {}
ormdic = {}
for line in f:
  if a== 0:
    a=100
    continue
  ls = line.strip().split("\t")
  g = ls[1].upper()
  if g not in gene_to_sym:continue
  g = gene_to_sym[g]
  if g not in hpodic:hpodic[g] = set()
  if g not in ormdic:ormdic[g] = set()
  hpodic[g].add(ls[2])
  ormdic[g].add(ls[8])
hposet = set()
ormset = set()
for h in hpodic:
  h1 = hpodic[h]
  for hh1 in h1:
    hposet.add(hh1)
for oe in ormdic:
  o1 = ormdic[oe]
  for hh1 in o1:
    ormset.add(hh1)
p = sorted(list(hposet))
o = sorted(list(ormset))
hponum = {}
ormnum = {}
for t in range(len(p)):
    hponum[p[t]] =t 
for t in range(len(o)):
    ormnum[o[t]] =t 

In [ ]:
#!wget https://maayanlab.cloud/static/hdfs/harmonizome/data/encodetfppi/gene_attribute_matrix.txt.gz
#引用元はHarmonizomeじゃなくてEncodeに変更
#!unzip gene_attribute_matrix.txt.gz
dft = pd.read_table('gene_attribute_matrix.txt')
dftf = dft.loc[2:].copy()
dftf.index=dftf["#"]
dftf=dftf.drop("#",axis=1)
dftf=dftf.drop("#.1",axis=1)
dftf=dftf.drop("GeneSym",axis=1)
dftf = dftf.astype('float16')
from tqdm.notebook import tqdm
tfdic = {}
for g in dftf.index:
  if g.upper() in gene_to_sym:
    tfdic[gene_to_sym[g.upper()]] = set()  
ind = dftf.index.tolist()
col = dftf.columns.tolist()
for n in tqdm(range(len(dftf))):
  tu = dftf.iloc[n].tolist()
  for nn in range(len(tu)):
    if tu[nn] == 1.0:
      if ind[n].upper() in gene_to_sym:
         tfdic[gene_to_sym[ind[n].upper()]].add("TF:"+col[nn])
tfnum={}
t = 0
for g in col:
  tfnum["TF:"+g] = t
  t += 1

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


  0%|          | 0/22819 [00:00<?, ?it/s]

In [ ]:
#Biogrid
#!wnget https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-4.2.193/BIOGRID-ALL-4.2.193.tab3.zip
#unzip BIOGRID-ALL-4.2.193.tab3.zip.1
f = open("BIOGRID-ALL-4.2.193.tab3.txt","r")
intdic={}
intnum={}
for line in f:
  ls = line.strip().split("\t")
  if ls[-2] == "Homo sapiens" and ls[-1] =="Homo sapiens":
      a = ls[7].upper()
      b = ls[8].upper()
      if a in gene_to_sym and b in gene_to_sym:
        a = gene_to_sym[a]
        b = gene_to_sym[b]
        if a not in intdic: intdic[a] =set()
        if b not in intdic: intdic[b] =set()
        intdic[a].add("BG:"+b)
        intdic[b].add("BG:"+a)
ints = sorted(["BG:"+i for i in list(intdic.keys())])
for n,ii in enumerate(ints):
  intnum[ii] = n

In [ ]:
#subcellular localization
#!wget https://www.proteinatlas.org/download/subcellular_location.tsv.zip
#!unzip subcellular_location.tsv.zip
import pandas as pd
df = pd.read_table('subcellular_location.tsv')
sl = list(df["Main location"])
sl2 = set()
for s in sl:
  if type(s) == float:
    continue
  s2  = s.strip().split(";")
  for ss2 in s2:
    sl2.add(ss2)
s = list(sl2)
s.sort()
slonum={}
for t in range(len(s)):
    slonum[s[t]] =t
geneslo = set(df["Gene name"])
slodic = {}
for g in geneslo:
    if g.upper() not in gene_to_sym:
      continue
    gu = gene_to_sym[g.upper()]
    try: 
            U = df[df["Gene name"]==g]
            tmp = list(U["Main location"])[0]
            if type(tmp) == float:
              continue
            slodic[gu] = set(tmp.strip().split(";"))
    except KeyError :
            print("KeyError:", UniId)
 

In [ ]:
#Expression
#!wget https://www.proteinatlas.org/download/rna_tissue_hpa.tsv.zip
#!unzip rna_tissue_hpa.tsv.zip
dfrna = pd.read_table("rna_tissue_hpa.tsv")
grouped = dfrna.groupby('Gene name')
medi = {}
sddi = {}
gm = grouped.mean()["TPM"].tolist()
gi = grouped.mean()["TPM"].index.tolist()
gs = grouped.std()["TPM"].tolist()
for n in range(len(gm)):
  medi[gi[n]] = gm[n]
  sddi[gi[n]] = gs[n]
from tqdm.notebook import tqdm
drg = dfrna["Gene name"].tolist()
drt = dfrna["TPM"]
sig = []
for n in tqdm(range(len(dfrna))):
  sig.append((drt[n]-medi[drg[n]])/(sddi[drg[n]]+0.001))
dfrna["sigma"]=sig
drti = dfrna["Tissue"].tolist()
tisdic = {}
for g in set(drg):
  if g.upper() in gene_to_sym:tisdic[gene_to_sym[g.upper()]] = set()

for n in tqdm(range(len(dfrna))):
  g = drg[n].upper()
  if g not in gene_to_sym:
    #print(g)
    continue
  t = drti[n]
  s = sig[n]
  u = gene_to_sym[g]
  if s >= 1.68: tisdic[u].add(t+":H5")
  if  0.91 <= s <  1.68: tisdic[u].add(t+":H4")
  if  0.53 <= s <  0.91: tisdic[u].add(t+":H3")
  if  0.28 <= s <  0.53: tisdic[u].add(t+":H2")
  if  0.09 <= s <  0.28: tisdic[u].add(t+":H1")
  if -0.16 <= s < -0.25: tisdic[u].add(t+":L1")
  if -0.25 <= s < -0.39: tisdic[u].add(t+":L2")
  if -0.39 <= s < -0.25: tisdic[u].add(t+":L3")
  if -0.65 <= s < -0.39: tisdic[u].add(t+":L4")
  if s < -0.65: tisdic[u].add(t+":L5")
tisnum = {}
c = 0
for t in sorted(list(set(drti))):
  for n in range(1,6):
    tisnum[t+":H"+str(n)]=c
    c +=1
  for n in range(1,6):
    tisnum[t+":L"+str(n)]=c
    c +=1

  0%|          | 0/845810 [00:00<?, ?it/s]

  0%|          | 0/845810 [00:00<?, ?it/s]

In [ ]:
ssig = sorted(sig)
ssigl = ssig[:ssig.index(0)]
ssigh = ssig[ssig.index(0):]
for i in range(6):
   print(ssigh[i*len(ssigh)//6])

0.0
0.09573919604237462
0.2828015689225128
0.5346948665116636
0.9176327162495056
1.6877739392289601


In [ ]:
dfdep = pd.read_csv("Cricpr.csv")
dfdep = dfdep.drop("DepMap_ID",axis=1)
dfdep = dfdep.dropna()
dfdep.columns = [d[:d.index("(")-1].upper() for d in dfdep.columns]
c = [a.upper() in gene_to_sym for a in dfdep.columns]
dfdep = dfdep.loc[:,c]
dfdep.columns = [gene_to_sym[a.upper()] for a in dfdep.columns]
deppp = {}
for d in tqdm(dfdep.columns):
  deppp[d] = np.array(dfdep[d])

  0%|          | 0/17624 [00:00<?, ?it/s]

In [ ]:
#word2vec
import pickle
with open("gene2vec.pickle","rb") as f:
  gene2vec = pickle.load(f)

In [ ]:
chrdic

#alldic,allnum

特徴量の一覧  
　知識ベース  
　　ggdic = GO, w2v  
　データベース  
    mgidic, slodic, hpodic, ormdic, intdic, tfdic, tisdic, chrdic, deppp

In [ ]:
#起動時間短縮のため、位置を決めてします
#For webapp
keys = ch
alldic = {}
for k in keys:
  alldic[k]=set()#初期化
def makedic(d):
  for k in keys:
    if k in d:
      alldic[k] = alldic[k] | d[k]
  return alldic
alldic = makedic(ggdic)
alldic = makedic(mgidic)
alldic = makedic(slodic)
alldic = makedic(hpodic)
alldic = makedic(ormdic)
alldic = makedic(intdic)
alldic = makedic(tfdic)
alldic = makedic(tisdic)
alldic = makedic(chrdic)
allnum = {}
def makenum(dn):
  a = len(allnum)
  for k in dn:
      allnum[k] = a+ dn[k]
  return allnum
allnum = makenum(gonum2)
allnum = makenum(mginum)
allnum = makenum(slonum)
allnum = makenum(hponum)
allnum = makenum(ormnum)
allnum = makenum(intnum)
allnum = makenum(tfnum)
allnum = makenum(tisnum)
allnum = makenum(chrnum)

In [ ]:
import pickle
with open("alldic_0909.pickle", "wb") as f:
  pickle.dump(alldic,f)
with open("allnum_0909.pickle", "wb") as f:
  pickle.dump(allnum,f)
#with open("model_0603.pickle", "wb") as f:
#  pickle.dump(model,f)

In [ ]:
with open("deppp_0909.pickle", "wb") as f:
  pickle.dump(deppp,f)

In [ ]:
import time 
time.sleep(3600)

#datadic, datanum

In [ ]:
#起動時間短縮のため、位置を決めてします
#For webapp
keys = ch
datdic = {}
for k in keys:
  datdic[k]=set()#初期化
def makedic(d):
  for k in keys:
    if k in d:
      datdic[k] = datdic[k] | d[k]
  return datdic
datdic = makedic(mgidic)
datdic = makedic(slodic)
datdic = makedic(hpodic)
datdic = makedic(intdic)
datdic = makedic(tfdic)
datdic = makedic(tisdic)
datdic = makedic(chrdic)
datnum = {}
def makenum(dn):
  a = len(datnum)
  for k in dn:
      datnum[k] = a+ dn[k]
  return datnum
datnum = makenum(gonum2)
datnum = makenum(mginum)
datnum = makenum(slonum)
datnum = makenum(hponum)
datnum = makenum(intnum)
datnum = makenum(tfnum)
datnum = makenum(tisnum)
datnum = makenum(chrnum)

In [ ]:
import pickle
with open("datdic_0909.pickle", "wb") as f:
  pickle.dump(datdic,f)
with open("datnum_0909.pickle", "wb") as f:
  pickle.dump(datnum,f)
#with open("model_0603.pickle", "wb") as f:
#  pickle.dump(model,f)

In [ ]:
len(datnum)
len(allnum)

54046